In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

### Load datasets

In [2]:
met = pd.read_csv('../input_data/moce5/moce5_raw_metdata.csv')
sat = pd.read_csv('../input_data/moce5/moce5_raw_skindata.csv')

### Extract wanted quantities

In [3]:
satday = sat['localday'].to_numpy(np.float64)
satlong = sat['long'].to_numpy(np.float64)
satlat = sat['lat'].to_numpy(np.float64)

metday = met['localday'].to_numpy(np.float64)
metlong = met['long'].to_numpy(np.float64)
metlat = met['lat'].to_numpy(np.float64)

longwave = met['long_wave'].to_numpy(np.float64)
shortwave = met['short_wave'].to_numpy(np.float64)
airtemp = sat['AirTemp'].to_numpy(np.float64)
precipitation = met['precipitation'].to_numpy(np.float64)
salinity = met['salinity'].to_numpy(np.float64)
relhumi = met['rel_humi'].to_numpy(np.float64)
boatspeed = met['speed_knots'].to_numpy(np.float64)
winddir = met['tdir'].to_numpy(np.float64)

### Conflate skin and met data

In [4]:
# Find elements of a dense sequence that are similar to the values of a sparse sequence
def get_near_indices(a,b):
    """
    Get indexes of values in array b that are nearest to the values in array a
    (The values in a and b must increase monotonically!)
    """
    c = np.zeros(a.shape[0])
    j = 0
    for i in range(a.shape[0]):
        while b[j] < a[i]:
            j += 1
        if np.abs(b[j-1]-a[i]) < np.abs(b[j]-a[i]):
            c[i] = j-1
        else:
            c[i] = j
    return c.astype(int)

In [5]:
# Get indices of metday corresponding to satday
metidx = get_near_indices(satday,metday)

# Clean data by removing corresponding points with >10min time difference
satidx_clean = np.where(np.abs(metday[metidx]-satday) < 10/24/60)
metidx_clean = metidx[satidx_clean]

# Clean data by removing corresponding data points with >0.05 degree difference in lat or long
pos_clean_idx = []
for i in range(satidx_clean[0].shape[0]):
    if np.abs(metlong[metidx_clean][i]-satlong[satidx_clean][i]) > 0.05 or np.abs(metlat[metidx_clean][i]-satlat[satidx_clean][i]) > 0.05:
        pos_clean_idx.append(i)
    else:
        pass
sidx = np.delete(satidx_clean[0],np.array(pos_clean_idx))
midx = metidx[sidx]



In [6]:
lon_raw = metlong[midx]
lat_raw = metlat[midx]


### Get cleaned, conflated quantities

In [7]:
skintemp_raw = sat['skintemp'].to_numpy(np.float64)[sidx]
skintemperr_raw = sat['StdDevSkinTemp'].to_numpy(np.float64)[sidx]
wind_raw = met['tspd'].to_numpy(np.float64)[midx]
ftemp_raw = met['sea_temp_corr'].to_numpy(np.float64)[midx]
hourday_raw = satday[sidx] % 1
times_raw = satday[sidx]

longwave_raw = longwave[midx]
shortwave_raw = shortwave[midx]
precip_raw = precipitation[midx]
sal_raw = salinity[midx]
relhumi_raw = relhumi[midx]
airtemp_raw = airtemp[sidx]
boatspeed_raw = boatspeed[midx]
winddir_raw = winddir[midx]

# clean by removing part where ftemp_raw is -99 degrees
ftemp = ftemp_raw[ftemp_raw>0] + 273.15
skintemp = skintemp_raw[ftemp_raw>0]
skintemperr = skintemperr_raw[ftemp_raw>0]
wind = wind_raw[ftemp_raw>0]
wdir = winddir_raw[ftemp_raw>0]

hourday = hourday_raw[ftemp_raw>0]
times = times_raw[ftemp_raw>0]
lon = lon_raw[ftemp_raw>0]
lat = lat_raw[ftemp_raw>0]

lwave = longwave_raw[ftemp_raw>0]
swave = shortwave_raw[ftemp_raw>0]
precip = precip_raw[ftemp_raw>0]
sal = sal_raw[ftemp_raw>0]
humi = relhumi_raw[ftemp_raw>0]
atemp = airtemp_raw[ftemp_raw>0]
bspeed = boatspeed_raw[ftemp_raw>0]

In [8]:
T_f = np.mean(ftemp)
humid = np.ones(len(times))*15e-3 # arbitrarily assume a specific humidity of 15 g/kg (since it is not given in the dataset)
atemp_rel = atemp - ftemp + T_f
dsst = skintemp - ftemp
ssterr = 2*skintemperr* (1 + bspeed/np.amax(bspeed))
boatspd = bspeed * 0.51444444

time = (times-np.floor(times[0])) * 86400
time_day = times-np.floor(times[0])
day = np.floor(times).astype(int)

## Export data

In [9]:
ds = xr.Dataset(
    data_vars=dict(
        dsst=(['time'], dsst),
        dsst_err=(['time'], ssterr),
        skinsst=(['time'], skintemp),
        ftemp=(['time'], ftemp),
        atemp=(['time'], atemp),
        atemp_rel=(['time'], atemp_rel),
        swrad=(['time'], lwave),
        wind=(['time'], wind),
        wind_dir=(['time'], wdir),
        humid=(['time'], humid),
        time_day=(['time'], time_day),
        day_of_year=(['time'], day),
        lat=(['time'], lat),
        lon=(['time'], lon),
        boatspd=(['time'], boatspd),
    ),
    coords={
        'time': time,
        'T_f': T_f,
    },
    attrs=dict(
        Description='Dataset from the MOCE-5 cruise in the Eastern Pacific and Gulf of California',
        Cruise_dates='01-21 October 1999',
        Ship_name='R/V Melville',
        Notes='This data set has been processed to be used as input data for the diuSST model (github.com/reykboerner/diusst).\nSpecific humidity data is missing in the original data set and has here been substituted by the constant value humid=15 g/kg.\natemp_rel is calculated as (atemp - ftemp + T_f), where T_f is the mean of ftemp.\nThe standard error of dsst is estimated based on the standard deviation of skinsst measurements and the boat speed.\n',
        Data_source='Raw data kindly provided by Prof. Peter Minnett, University of Miami',
        Contact='Reyk Börner (reyk.boerner@reading.ac.uk)',
        Date='08 July 2022',
    ),
)

ds.time.attrs['units']='seconds'
ds.time.attrs['long_name']='Local sun time in seconds since 00h00 midnight on 01 October 1999'
ds.dsst.attrs['units']='Kelvin'
ds.dsst.attrs['long_name']='Diurnal warming (skinsst minus ftemp)'
ds.dsst_err.attrs['units']='Kelvin'
ds.dsst_err.attrs['long_name']='Standard error of dsst (see Notes)'
ds.skinsst.attrs['units']='Kelvin'
ds.skinsst.attrs['long_name']='Radiometric skin SST'
ds.ftemp.attrs['units']='Kelvin'
ds.ftemp.attrs['long_name']='Sea temperature at 3m depth'
ds.atemp.attrs['units']='Kelvin'
ds.atemp.attrs['long_name']='Air temperature'
ds.atemp_rel.attrs['units']='Kelvin'
ds.atemp_rel.attrs['long_name']='Air temperature relative to mean foundation temperature (see Notes)'
ds.swrad.attrs['units']='Watt per square meter'
ds.swrad.attrs['long_name']='Incident downward shortwave irradiance'
ds.wind.attrs['units']='meter per second'
ds.wind.attrs['long_name']='Horizontal wind speed'
ds.wind_dir.attrs['units']='degree'
ds.wind_dir.attrs['long_name']='Wind direction'
ds.humid.attrs['units']='kilogram per kilogram'
ds.humid.attrs['long_name']='Specific humidity (set to constant value 0.015 due to missing data, see Notes)'
ds.time_day.attrs['units']='days'
ds.time_day.attrs['long_name']='Local sun time in days since 00h00 midnight on 01 October 1999'
ds.day_of_year.attrs['units']='days'
ds.day_of_year.attrs['long_name']='Day of the year 1999'
ds.lat.attrs['units']='degrees north'
ds.lat.attrs['long_name']='Latitude'
ds.lon.attrs['units']='degrees east'
ds.lon.attrs['long_name']='Longitude'
ds.boatspd.attrs['units']='meter per second'
ds.boatspd.attrs['long_name']='Cruise speed of vessel'
ds.T_f.attrs['units']='Kelvin'
ds.T_f.attrs['long_name']='Mean foundation temperature (mean of ftemp)'

In [10]:
ds.to_netcdf('../input_data/moce5/moce5_dataset.cdf')